In [ ]:
import string

logo = """             
            88             88                                 
           ""             88                                 
                          88                                 
 ,adPPYba, 88 8b,dPPYba,  88,dPPYba,   ,adPPYba, 8b,dPPYba,  
a8"     "" 88 88P'    "8a 88P'    "8a a8P_____88 88P'   "Y8  
8b         88 88       d8 88       88 8PP""""""" 88          
"8a,   ,aa 88 88b,   ,a8" 88       88 "8b,   ,aa 88          
 `"Ybbd8"' 88 88`YbbdP"'  88       88  `"Ybbd8"' 88          
              88                                             
              88           
"""

print(logo)

import string
import numpy as np

class CaesarCipher:
    def __init__(self):
        self.character_set = string.ascii_lowercase

    def caesar(self, start_text, shift_amount, cipher_direction):
        end_text = ""
        shift_amount = shift_amount % len(self.character_set)
        if cipher_direction == "decode":
            shift_amount *= -1

        for char in start_text:
            if char in self.character_set:
                position = self.character_set.index(char)
                new_position = (position + shift_amount) % len(self.character_set)
                end_text += self.character_set[new_position]
            else:
                end_text += char
        return end_text

class VigenereCipher:
    def __init__(self):
        self.character_set = string.ascii_lowercase

    def vigenere(self, start_text, key, cipher_direction):
        end_text = ""
        key = key.lower()
        key_length = len(key)
        key_as_int = [self.character_set.index(char) for char in key]
        text_as_int = [self.character_set.index(char) for char in start_text]
        for i in range(len(text_as_int)):
            if cipher_direction == "encode":
                value = (text_as_int[i] + key_as_int[i % key_length]) % len(self.character_set)
            elif cipher_direction == "decode":
                value = (text_as_int[i] - key_as_int[i % key_length]) % len(self.character_set)
            end_text += self.character_set[value]
        return end_text

class AdditionCipher:
    def __init__(self):
        self.character_set = string.ascii_lowercase

    def addition(self, start_text, shift_amount, cipher_direction):
        return CaesarCipher().caesar(start_text, shift_amount, cipher_direction)

class MultiplicativeCipher:
    def __init__(self):
        self.character_set = string.ascii_lowercase

    def multiplicative(self, start_text, shift_amount, cipher_direction):
        end_text = ""
        shift_amount = shift_amount % len(self.character_set)
        if cipher_direction == "decode":
            shift_amount = self.mod_inverse(shift_amount, len(self.character_set))

        for char in start_text:
            if char in self.character_set:
                position = self.character_set.index(char)
                new_position = (position * shift_amount) % len(self.character_set)
                end_text += self.character_set[new_position]
            else:
                end_text += char
        return end_text

    def mod_inverse(self, a, m):
        a = a % m
        for x in range(1, m):
            if (a * x) % m == 1:
                return x
        return None

class PlayfairCipher:
    def __init__(self):
        self.character_set = string.ascii_lowercase.replace('j', '')

    def generate_key_square(self, key):
        key = ''.join(sorted(set(key), key=lambda x: key.index(x)))
        alphabet = ''.join([c for c in self.character_set if c not in key])
        key_square = np.array(list(key + alphabet)).reshape(5, 5)
        return key_square

    def find_position(self, char, key_square):
        position = np.where(key_square == char)
        return position[0][0], position[1][0]

    def playfair(self, start_text, key, cipher_direction):
        key_square = self.generate_key_square(key)
        start_text = start_text.replace('j', 'i')
        text_pairs = []
        i = 0
        while i < len(start_text):
            a = start_text[i]
            if i + 1 < len(start_text):
                b = start_text[i + 1]
            else:
                b = 'x'
            if a == b:
                b = 'x'
                i += 1
            else:
                i += 2
            text_pairs.append((a, b))
        
        end_text = ''
        for a, b in text_pairs:
            row_a, col_a = self.find_position(a, key_square)
            row_b, col_b = self.find_position(b, key_square)
            if row_a == row_b:
                if cipher_direction == "encode":
                    end_text += key_square[row_a, (col_a + 1) % 5] + key_square[row_b, (col_b + 1) % 5]
                else:
                    end_text += key_square[row_a, (col_a - 1) % 5] + key_square[row_b, (col_b - 1) % 5]
            elif col_a == col_b:
                if cipher_direction == "encode":
                    end_text += key_square[(row_a + 1) % 5, col_a] + key_square[(row_b + 1) % 5, col_b]
                else:
                    end_text += key_square[(row_a - 1) % 5, col_a] + key_square[(row_b - 1) % 5, col_b]
            else:
                end_text += key_square[row_a, col_b] + key_square[row_b, col_a]
        return end_text

class MenuDrivenCipher:
    def __init__(self):
        self.caesar_cipher = CaesarCipher()
        self.vigenere_cipher = VigenereCipher()
        self.addition_cipher = AdditionCipher()
        self.multiplicative_cipher = MultiplicativeCipher()
        self.playfair_cipher = PlayfairCipher()

    def run(self):
        while True:
            print("\nMenu:")
            print("1. Caesar Cipher")
            print("2. Vigenère Cipher")
            print("3. Addition Cipher")
            print("4. Multiplicative Cipher")
            print("5. Playfair Cipher")
            print("6. Exit")
            choice = input("Enter your choice: ")

            if choice == "1":
                self.run_caesar()
            elif choice == "2":
                self.run_vigenere()
            elif choice == "3":
                self.run_addition()
            elif choice == "4":
                self.run_multiplicative()
            elif choice == "5":
                self.run_playfair()
            elif choice == "6":
                print("Goodbye!")
                break
            else:
                print("Invalid choice. Please try again.")

    def run_caesar(self):
        direction = input("Type 'encode' to encrypt, type 'decode' to decrypt:\n").lower()
        text = input("Type your message:\n").lower()
        shift = int(input("Type the shift number:\n"))
        result = self.caesar_cipher.caesar(start_text=text, shift_amount=shift, cipher_direction=direction)
        print(f"Here's the {direction}d result: {result}")

    def run_vigenere(self):
        direction = input("Type 'encode' to encrypt, type 'decode' to decrypt:\n").lower()
        text = input("Type your message:\n").lower()
        key = input("Type the key:\n").lower()
        result = self.vigenere_cipher.vigenere(start_text=text, key=key, cipher_direction=direction)
        print(f"Here's the {direction}d result: {result}")

    def run_addition(self):
        direction = input("Type 'encode' to encrypt, type 'decode' to decrypt:\n").lower()
        text = input("Type your message:\n").lower()
        shift = int(input("Type the shift number:\n"))
        result = self.addition_cipher.addition(start_text=text, shift_amount=shift, cipher_direction=direction)
        print(f"Here's the {direction}d result: {result}")

    def run_multiplicative(self):
        direction = input("Type 'encode' to encrypt, type 'decode' to decrypt:\n").lower()
        text = input("Type your message:\n").lower()
        shift = int(input("Type the shift number:\n"))
        result = self.multiplicative_cipher.multiplicative(start_text=text, shift_amount=shift, cipher_direction=direction)
        print(f"Here's the {direction}d result: {result}")

    def run_playfair(self):
        direction = input("Type 'encode' to encrypt, type 'decode' to decrypt:\n").lower()
        text = input("Type your message:\n").lower()
        key = input("Type the key:\n").lower()
        result = self.playfair_cipher.playfair(start_text=text, key=key, cipher_direction=direction)
        print(f"Here's the {direction}d result: {result}")

if __name__ == "__main__":
    menu = MenuDrivenCipher()
    menu.run()



             
            88             88                                 
           ""             88                                 
                          88                                 
 ,adPPYba, 88 8b,dPPYba,  88,dPPYba,   ,adPPYba, 8b,dPPYba,  
a8"     "" 88 88P'    "8a 88P'    "8a a8P_____88 88P'   "Y8  
8b         88 88       d8 88       88 8PP" 88          
"8a,   ,aa 88 88b,   ,a8" 88       88 "8b,   ,aa 88          
 `"Ybbd8"' 88 88`YbbdP"'  88       88  `"Ybbd8"' 88          
              88                                             
              88           


Menu:
1. Caesar Cipher
2. Vigenère Cipher
3. Addition Cipher
4. Multiplicative Cipher
5. Playfair Cipher
6. Exit
Enter your choice: 4
Type 'encode' to encrypt, type 'decode' to decrypt:
encode
Type your message:
heeloo
Type the shift number:
23
Here's the encoded result: footkk

Menu:
1. Caesar Cipher
2. Vigenère Cipher
3. Addition Cipher
4. Multiplicative Cipher
5. Playfair Cipher
6. Exit
